# Cleaning and Preprocessing 
### Summary of Activity

- **Load and Clean Data**:  
  We load data for benchmark index prices and constituent prices from CSV files, ensuring numeric columns are properly formatted and the `Date` column is converted to datetime.
  
- **Data Cleaning**:  
  This data originates from the Bloomberg terminal and uses BBG tickers. We remove unwanted suffixes (e.g., " Equity") from column names for clarity  
  We reshape the data using `melt` to create a tidy format with columns for `Date`, `Ticker`, and `Price`.
  
- **Date Range Filtering**:  
  We filter the data to include only rows within a specified date range (e.g.`1 Jan 2015` to `30 Sep 2024`).

- **Return Calculation**:  
  We extend the dataframe by calculating returns for various periods (1 business day, 1 week, 1 month, 1 quarter, and 1 year), handling missing values using a lookback window of 5 business days.
  
- **Data Merging**:  
  We merge the benchmark and constituent data to compute active returns by subtracting benchmark returns from constituent returns.

- **Validation**:  
  We verify that the active returns sum correctly by checking if the `active_returns` plus benchmark returns equals the total return.

- **Thresholding**:  
  We apply a threshold to active returns by converting them to absolute values and then comparing each value to a specified threshold in basis points (e.g., 100 bps). 
  
  If the absolute active return exceeds the threshold, it is marked as `True`; otherwise, it is marked as `False`. This thresholded data helps identify significant active returns relative to a set performance benchmark, and will be used for our classification task.
  

- **Final Output**:  
  We save the cleaned, processed, and thresholded active returns to CSV files for further analysis.


In [ ]:
import numpy as np
import pandas as pd
import time
from pandas.tseries.offsets import BDay, Week, MonthEnd, QuarterEnd, YearEnd
from functions.constants import BM_NAME, STARTDATE, ENDDATE, N_THRESHOLD_BPS, DATA_DIR

In [2]:
# Load data
bm_index_prices_df_path = DATA_DIR + BM_NAME + "_BM_prices.csv"
bm_holdings_prices_df_path = DATA_DIR + BM_NAME + "_constituents_prices.csv"

def load_bm_index_df(path):
    df = pd.read_csv(path, skiprows=3)
    df = df.rename(columns={df.columns[0]: "Date"})
    df = df[2:]
    #ensure all cols except Date are numeric. Coerce nan if not
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df["Date"] = pd.to_datetime(df["Date"])
    return df

bm_index_prices = load_bm_index_df(bm_index_prices_df_path)
bm_holdings_prices = load_bm_index_df(bm_holdings_prices_df_path)
print("Raw data: BM prices")
print(bm_index_prices.head())
print("Raw data: BM holdings prices")
print(bm_holdings_prices.head())

/var/folders/7n/wd610q_96_5crpp9w0bq0gfh0000gn/T/ipykernel_93499/1015973410.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

Raw data: BM prices
        Date  SPX Index
2 2024-09-30    5762.48
3 2024-09-27    5738.17
4 2024-09-26    5745.37
5 2024-09-25    5722.26
6 2024-09-24    5732.93
Raw data: BM holdings prices
        Date  A UN Equity  AAPL UW Equity  ABBV UN Equity  ABNB UW Equity  \
2 2024-09-30       148.48          233.00          197.48          126.81   
3 2024-09-27       148.06          227.79          194.79          128.25   
4 2024-09-26       146.74          227.52          191.90          129.25   
5 2024-09-25       139.44          226.37          191.26          130.71   
6 2024-09-24       141.87          227.37          193.51          131.48   

   ABT UN Equity  ACGL UW Equity  ACN UN Equity  ADBE UW Equity  \
2         114.01          111.88         353.48          517.78   
3         112.43          113.41         349.70          515.48   
4         112.73          112.57         355.81          515.38   
5         112.61          113.07         337.05          517.90   
6        

In [ ]:
def remove_BBG_suffixes(df):
    """
    Remove the suffixes " Equity" from the column names
    """
    df.columns = df.columns.str.replace(' Equity', '')
    # df.columns = df.columns.str.replace(' Index', '') #let's keep the SPX Index in for clarity
    return df

def melt_data(df):
    """
    Melt the dataframe to have a column for the date, a column for the ticker and a column for the price
    """
    df = df.melt(id_vars=["Date"], var_name="Ticker", value_name="Price")
    return df

def filter_date_range(df, start_date=STARTDATE, end_date=ENDDATE):
    """
    Filter the dataframe to keep only the rows within the specified date range
    """
    df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    return df

def extend_with_returns(df, periods=["1b"], lookback_window='5b'):
    '''
    Extend the input dataframe with additional return columns for specified periods.
    Handles holidays and missing data by looking back up to a defined number of business days.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The input dataframe containing at least 'Date', 'Ticker', and 'Price' columns.
    periods : list, optional, default=["1b"]
        A list of periods for which returns are calculated. Supported values include:
        '1b' (1 business day), '1w' (1 week), '1m' (1 month), '1q' (1 quarter), '1y' (1 year), 
        NOT YET SUPPORTED: 'mtd' (month-to-date), 'qtd' (quarter-to-date), 'ytd' (year-to-date).
    lookback_window : str, optional, default='5b'
        The lookback window for calculating returns when frequency-based returns are unavailable. 
        The default is 5 business days.

    Returns:
    --------
    pandas.DataFrame
        A new dataframe with additional columns for each specified return period (e.g., 'returns_1b', 'returns_mtd').
    '''
    # Ensure Date column is datetime and set it as index
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    
    # Sort the dataframe by Date and Ticker
    df = df.sort_index().sort_values('Ticker')
    
    # Define offset mappings for supported periods
    offset_map = {
        '1b': BDay(1),
        '1w': Week(1),
        '1m': MonthEnd(1),
        '1q': QuarterEnd(1),
        '1y': YearEnd(1)
    }
    # Extract number of business days from the lookback_window
    lookback_periods = int(lookback_window[:-1])  # Remove the 'b' and convert to int
    lookback_offset = BDay(lookback_periods)

    for period in periods:
        t = time.time()
        if period in offset_map:
            offset = offset_map[period]
            # Calculate returns based on standard periods
            df[f'returns_{period}'] = df.groupby('Ticker')['Price'].apply(
                lambda x: x.pct_change(freq=offset).fillna(x.pct_change(freq=lookback_offset))
            )
        else:
            # For non-standard periods, apply lookback window logic
            df[f'returns_{period}'] = df.groupby('Ticker')['Price'].apply(
                lambda x: x.pct_change(periods=lookback_periods).fillna(x.pct_change(periods=lookback_periods))
            )
        # Handling missing longer returns with a rolling window (e.g., '1w', '1m')
        if period in ['1w', '1m', '1q', '1y']:
            df[f'returns_{period}'] = df.groupby('Ticker')[f'returns_{period}'].fillna(method='ffill')
        print("Calculating returns for period:", period, "took", time.time() - t, "seconds")
    
    df = df.reset_index()
    return df

def preprocess_and_clean_data(df):
    t = time.time()
    df = remove_BBG_suffixes(df)
    print(f"Time taken to remove BBG suffixes: {time.time() - t:.6f} seconds")
    t = time.time()
    df = melt_data(df)
    print(f"Time taken to melt data: {time.time() - t:.6f} seconds")
    t = time.time()
    df = extend_with_returns(df, periods=["1b","1w", "1m", "1q", "1y"])
    print(f"Time taken to extend with returns: {time.time() - t:.6f} seconds")
    t = time.time()
    df = filter_date_range(df)
    print(f"Time taken to filter date range: {time.time() - t:.6f} seconds")
    return df

bm_index_prices_preproc = preprocess_and_clean_data(bm_index_prices)
print("Preprocessed data: BM prices")
print(bm_index_prices_preproc.head())
bm_holdings_prices_preproc = preprocess_and_clean_data(bm_holdings_prices)
print("Preprocessed data: BM holdings prices")
print(bm_holdings_prices_preproc.head())

Preprocessed data: BM prices
        Date     Ticker    Price  returns_1b  returns_1w  returns_1m  \
1 2021-02-24  SPX Index  3925.43    0.011352   -0.001501   -0.001501   
2 2021-02-25  SPX Index  3829.34   -0.024479   -0.021623   -0.021623   
3 2021-02-26  SPX Index  3811.15   -0.004750   -0.024460   -0.024460   
4 2021-03-01  SPX Index  3901.82    0.023791    0.006532    0.006532   
5 2021-03-02  SPX Index  3870.29   -0.008081   -0.002855   -0.002855   

   returns_1q  returns_1y  
1   -0.001501   -0.001501  
2   -0.021623   -0.021623  
3   -0.024460   -0.024460  
4    0.006532    0.006532  
5   -0.002855   -0.002855  
Preprocessed data: BM holdings prices
        Date Ticker   Price  returns_1b  returns_1w  returns_1m  returns_1q  \
0 2024-09-30   A UN  148.48    0.002837    0.062165    0.075708    0.125104   
2 2024-07-18   A UN  131.97   -0.011461    0.008791    0.008791    0.008791   
3 2015-03-13   A UN   40.87   -0.005596   -0.015892   -0.015892   -0.015892   
4 2023-02-14   A

In [4]:
#print percentage of missing in each column
print("After having handled missing values due to holidays and weekends:\n")
print("Fraction of missing values in BM index prices:")
print(bm_index_prices_preproc.isnull().mean())
print("Fraction of missing values in BM holdings prices:")
print(bm_holdings_prices_preproc.isnull().mean())

After having handled missing values due to holidays and weekends:

Fraction of missing values in BM index prices:
Date          0.0
Ticker        0.0
Price         0.0
returns_1b    0.0
returns_1w    0.0
returns_1m    0.0
returns_1q    0.0
returns_1y    0.0
dtype: float64
Fraction of missing values in BM holdings prices:
Date          0.000000
Ticker        0.000000
Price         0.061841
returns_1b    0.000292
returns_1w    0.000280
returns_1m    0.000279
returns_1q    0.000279
returns_1y    0.000279
dtype: float64


In [5]:
# TODO: MOVE THIS OVER TO HELPER_FNS.PY!

def get_active_returns(bm_index_prices, bm_holdings_prices):
    #establish a safe local scope
    bm_index_prices = bm_index_prices.copy()
    bm_holdings_prices = bm_holdings_prices.copy()

    holdings_cols = bm_holdings_prices.columns
    inferred_periods = [col for col in holdings_cols if 'returns' in col]
    inferred_periods = [col.split('_')[1] for col in inferred_periods]
    #rename bm_index_prices columns to have "bm_returns_" prefix. 
    bm_index_prices.columns = [col if col == "Date" else "bm_" + col for col in bm_index_prices.columns]
    # print(bm_index_prices.head())
    #merge the two dataframes on date left
    merged = pd.merge(bm_holdings_prices, bm_index_prices, on="Date", how="left")
    #calculate active returns
    for period in inferred_periods:
        merged["active_returns_" + period] = merged["returns_" + period] - merged["bm_returns_" + period]
    # print(merged.head())
    
    #TEST TO ENSURE THAT THE ACTIVE RETURNS ARE CORRECT AND SUMS TO WHEN ADDING BM RETURNS AND SUBTRACTING BM RETURNS
    merged_for_testing = merged.copy()
    merged_for_testing["TEST_COL"] = merged_for_testing["active_returns_1b"] + merged_for_testing["bm_returns_1b"] - merged_for_testing["returns_1b"]
    merged_for_testing["TEST_COL"] = merged_for_testing["TEST_COL"].fillna(0)  # Replace NaNs with zeros for comparison
    assert np.allclose(merged_for_testing["TEST_COL"], 0), "Active returns are not correctly calculated"

    #keep only (TICKER, Date, active_returns_*) columns
    active_returns = merged[["Ticker", "Date"] + [col for col in merged.columns if "active_returns" in col]]
    return active_returns

active_returns = get_active_returns(bm_index_prices_preproc, bm_holdings_prices_preproc)
active_returns

,Ticker,Date,active_returns_1b,active_returns_1w,active_returns_1m,active_returns_1q,active_returns_1y
0,A UN,2024-09-30,-0.001400,0.054486,0.031794,0.090025,0.054486
1,A UN,2024-07-18,-0.003644,0.015944,0.015944,0.015944,0.015944
2,A UN,2015-03-13,0.000479,-0.007269,-0.007269,-0.007269,-0.007269
3,A UN,2023-02-14,-0.009810,-0.002821,-0.002821,-0.002821,-0.002821
4,A UN,2018-02-20,0.020071,0.020071,0.020071,0.020071,0.020071
...,...,...,...,...,...,...,...
1233351,ZTS UN,2018-11-07,0.004988,0.018237,0.018237,0.018237,0.018237
1233352,ZTS UN,2023-04-27,-0.013726,-0.003927,-0.003927,-0.003927,-0.003927
1233353,ZTS UN,2016-08-23,-0.001561,-0.020143,-0.020143,-0.020143,-0.020143
1233354,ZTS UN,2016-08-22,-0.024370,-0.018526,-0.018526,-0.018526,-0.018526


In [6]:
active_returns_thresholded = active_returns.copy()
active_returns_thresholded = active_returns_thresholded.set_index(["Ticker", "Date"])
active_returns_thresholded = active_returns_thresholded > (N_THRESHOLD_BPS / 10000)
active_returns_thresholded = active_returns_thresholded.astype(float)
active_returns_thresholded = active_returns_thresholded.reset_index()
active_returns_thresholded

,Ticker,Date,active_returns_1b,active_returns_1w,active_returns_1m,active_returns_1q,active_returns_1y
0,A UN,2024-09-30,0.0,1.0,1.0,1.0,1.0
1,A UN,2024-07-18,0.0,1.0,1.0,1.0,1.0
2,A UN,2015-03-13,0.0,0.0,0.0,0.0,0.0
3,A UN,2023-02-14,0.0,0.0,0.0,0.0,0.0
4,A UN,2018-02-20,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
1233351,ZTS UN,2018-11-07,0.0,1.0,1.0,1.0,1.0
1233352,ZTS UN,2023-04-27,0.0,0.0,0.0,0.0,0.0
1233353,ZTS UN,2016-08-23,0.0,0.0,0.0,0.0,0.0
1233354,ZTS UN,2016-08-22,0.0,0.0,0.0,0.0,0.0


In [7]:
active_returns_path = DATA_DIR + BM_NAME + "_active_returns.csv"
active_returns.to_csv(active_returns_path)
print("Active returns saved to " + active_returns_path)
active_returns_thresholded_path = DATA_DIR + BM_NAME + "_active_returns_thresholded_" + str(N_THRESHOLD_BPS) + "bps.csv"
active_returns_thresholded.to_csv(active_returns_thresholded_path)
print("Active returns thresholded saved to " + active_returns_thresholded_path)

Active returns saved to ./../data/SP500_active_returns.csv
Active returns thresholded saved to ./../data/SP500_active_returns_thresholded_100bps.csv
